<center>
<img src="../../img/ods_stickers.jpg">
## Открытый курс по машинному обучению
<center>Автор материала: Плаксина Елена Константиновна, Levka.

## <center>Обзор библиотеки для генерации временных признаков tsfresh</center>
### <center>Time Series FeatuRe Extraction based on Scalable Hypothesis tests</center>

Библиотека используется для извлечения признаков из временных рядов. Практически все признаки, которые могут прийти вам в голову, уже внесены в расчёт этой библиотеки и нет никакого смысла создавать их самому, когда это можно сделать парой строчек кода из библиотеки.

Извлечённые признаки могут быть использованы для описания или кластеризации временных рядов. Также их можно использовать для задач классификации/регрессии на временных рядах.

<img src="http://tsfresh.readthedocs.io/en/latest/_images/introduction_ts_exa_features.png" width="700" height="600">

### Процесс расчёта признаков состоит из двух этапов:
- Расчёт всех возможных признаков

```python
from tsfresh import extract_features
extracted_features = extract_features(timeseries, column_id="id", column_sort="time")
```
- Отбор релевантных признаков и удаление константных/нулевых признаков

```python
from tsfresh import select_features
from tsfresh.utilities.dataframe_functions import impute

impute(extracted_features)  # удаление константных признаков
features_filtered = select_features(extracted_features, y)  # отбор признаков
```

### Процедура отбора признаков
#### Стадия 1
Расчёт признаков
#### Стадия 2
Проверка на значимость каждого признака, расчёт p-value
#### Стадия 3
Поправка на множественную проверку гипотез Бенджамини-Иекутиели

<img src="http://tsfresh.readthedocs.io/en/latest/_images/feature_extraction_process_20160815_mc_1.png" width="700" height="600">

### Приведём пример генерации признаков на основе датасета Human Activity Recognition

In [ ]:
import matplotlib.pylab as plt

%matplotlib inline
import numpy as np
import pandas as pd
import seaborn as sns
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from tsfresh import (extract_features, extract_relevant_features,
                     select_features)
from tsfresh.examples.har_dataset import (download_har_dataset,
                                          load_har_classes, load_har_dataset)
from tsfresh.feature_extraction import ComprehensiveFCParameters
from tsfresh.utilities.dataframe_functions import impute

**Загрузка и отрисовка данных**

In [ ]:
download_har_dataset()

In [ ]:
df = load_har_dataset()

In [ ]:
plt.title('accelerometer reading')
plt.plot(df.iloc[0,:])
plt.show()

**Извлечение признаков**

In [ ]:
# расчёт только определённого набора параметров, заданного в ComprehensiveFCParameters
extraction_settings = ComprehensiveFCParameters()

In [ ]:
# переформируем данные 500 первых показаний сенсоров column-wise, как этого требует формат библиотеки
N = 500
master_df = pd.DataFrame({0: df[:N].values.flatten(),
                          1: np.arange(N).repeat(df.shape[1])})
master_df.head()

In [ ]:
X = extract_features(master_df, column_id=1, impute_function=impute, default_fc_parameters=extraction_settings)

In [ ]:
"Число рассчитанных признаков: {}.".format(X.shape[1])

**Обучение классификатора**

In [ ]:
y = load_har_classes()[:N]
y.shape

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.2)

In [ ]:
cl = DecisionTreeClassifier()
cl.fit(X_train, y_train)
print(classification_report(y_test, cl.predict(X_test)))

**Отберём признаки для каждого класса отдельно и решим задачу бинарной классификации**

In [ ]:
relevant_features = set()

for label in y.unique():
    y_train_binary = y_train == label
    X_train_filtered = select_features(X_train, y_train_binary)
    print("Number of relevant features for class {}: {}/{}".format(label, X_train_filtered.shape[1], X_train.shape[1]))
    relevant_features = relevant_features.union(set(X_train_filtered.columns))

In [ ]:
len(relevant_features)

Мы уменьшили количество признаков с 794 до 264.

In [ ]:
X_train_filtered = X_train[list(relevant_features)]
X_test_filtered = X_test[list(relevant_features)]

In [ ]:
X_train_filtered.shape, X_test_filtered.shape

In [ ]:
cl = DecisionTreeClassifier()
cl.fit(X_train_filtered, y_train)
print(classification_report(y_test, cl.predict(X_test_filtered)))

Качество модели практически не изменилось, однако модель стала намного проще.

**Сравнение с классификатором на стандартных признаках**

In [ ]:
X_1 = df.iloc[:N,:]
X_1.shape

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X_1, y, test_size=.2)

In [ ]:
cl = DecisionTreeClassifier()
cl.fit(X_train, y_train)
print(classification_report(y_test, cl.predict(X_test)))

Как видимо, качество модели значительно улучшилось по сравнению с наивным классификатором.